# Tensorflow Estimator [reference](https://ai.google/research/pubs/pub46369)

##### This file is converted/produced from [TensorFlow Estimators: Managing Simplicity vs. Flexibility inHigh-Level Machine Learning Frameworks](https://arxiv.org/pdf/1708.02637.pdf)

Tensorflow Estimator consists of only four methods:traintrains the model, given training data.evaluatecomputes evaluation metrics over test data,predictperforms inference on new data given a trained model, and finally,exportsavedmodelexports a SavedModel, a serialization formatwhich allows the model to be used in TensorFlow Serving, a prebuiltproduction server for TensorFlow models.

1. the user confgures an Estimatorby passing a callback, the <b>model_fn</b>, to the constructor.  When one of its methods is called, Estimatorcreates a TensorFlow graph, sets up the input pipelinespecifed by the user in the arguments to the method,and then calls themodelfnwith appropriate arguments to gener-ate the graph representing the model.

2. Estimatorhides some TensorFlow concepts, such asGraphandSession, from the user.

3.  the Estimatorconstructor also receives a confguration object calledRunConfigwhich communicates every-thing that thisEstimatorneeds to know about the environment.

4. in which the model will be run: how many workers are available,how often to save intermediate checkpoints.

5. Estimatorcreates a new graph, andpossibly restores from checkpoint, every time a method is called.

![Estimator Flow](./Basic_TF_Estimator_presentation/Estimator_flow.png)

### Specifying inputs with inputfn
Methodstrain,evaluate,andpredictall take an input function, which is expected to pro-duce two dictionaries:  
<b>one containingTensors with inputs (fea-tures)</b>, 
and <b>one containingTensors with labels</b>

### Specifying the model with modelfn
themodelfn, which returns opsfor training, evaluation, or prediction, depending on which graphis being requested

Conceptually, three entirely different graphs can be built, and different information is returned, depending on the mode parameterrepresenting the called method

To work with the estimators, we’ve to become comfortable with two new concepts
1. Feature columns:
    Feature columns are specifications for how the model should interpret the input data
    
2. Input Function:
    Input functions are used for passing input data to the model for training and evaluation
    
Basically workflow how to use [tensorflow estimator](https://medium.com/learning-machine-learning/introduction-to-tensorflow-estimators-part-1-39f9eb666bc7) including some step:
1. Loading the libraries and dataset.
2. Data proprocessing.
3. Defining the feature columns.
4. Building input function.
5. Model instantiation, training and evaluation.
6. Generating prediction.
7. Visualizing the model and the loss metrics using Tensorboard.

![feature_columns](./Basic_TF_Estimator_presentation/feature_columns.png)

<b>The code below will re-implemented the [tf.estimator tutorial](https://guillaumegenthial.github.io/introduction-tensorflow-estimator.html) from [Guillaume Genthial blog](https://guillaumegenthial.github.io/)</b>

## References
[Understanding the Tensorflow Estimator API](https://www.lighttag.io/blog/tensorflow-estimator-api/)

[TensorFlow Estimators: Managing Simplicity vs. Flexibility inHigh-Level Machine Learning Frameworks](https://arxiv.org/pdf/1708.02637.pdf)

# Global configuration


In [1]:
import logging
from pathlib import Path
import sys

import tensorflow as tf

# Setup logging
Path('results').mkdir(exist_ok=True)
tf.logging.set_verbosity(logging.INFO)
handlers = [
    logging.FileHandler('results/main.log'),
    logging.StreamHandler(sys.stdout)
]
logging.getLogger('tensorflow').handlers = handlers


# Model
We do regression function f(x1, x2) = 2x
The model will learn to define two parameter a, b in linear model a * x1 + b * x2

so fisrt we construct model_fn

# Feed data with tf.data

1. The standard technique was to use a tf.placeholder that was updated through the run method of a tf.Session object. There was also an attempt of a more optimized input pipeline with threadings and queues.

2. A better (and almost perfect) way of feeding data to your tensorflow model is to use a wonderful new tensorflow API called tf.data


In [2]:
def generator_fn():
    for digit in range(2):
        line = 'I am digit {}'.format(digit)
        words = line.split()
        yield [w.encode() for w in words], len(words)
      
for words in generator_fn():
    print(words)


([b'I', b'am', b'digit', b'0'], 4)
([b'I', b'am', b'digit', b'1'], 4)


In [3]:
shapes = ([None], ())
types = (tf.string, tf.int32)

dataset = tf.data.Dataset.from_generator(generator_fn, output_shapes=shapes, output_types=types)


From /home/dat/anaconda3/envs/py35/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.py:429: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    


In [4]:
iterator = dataset.make_one_shot_iterator()
node = iterator.get_next()
with tf.Session() as sess:
    print(sess.run(node))
    print(sess.run(node))

(array([b'I', b'am', b'digit', b'0'], dtype=object), 4)
(array([b'I', b'am', b'digit', b'1'], dtype=object), 4)


# Train the Estimator
To train our model, we generate fake data


In [5]:
def train_generator_fn():
    for number in range(100):
        yield [number, number] [2 * number]
        
def train_input_fn():
    shapes, types = (2, 1), (tf.float32, tf.float32)
    dataset = tf.data.Dataset.from_generator(train_generator_fn, output_types=types, output_shapes = shapes)
    dataset = dataset.batch(20).repeat(200)
    return dataset


In [6]:
estimator = tf.estimator.Estimator(model_fn, "model", params={})
estimator.train(train_input_fn)

NameError: name 'model_fn' is not defined